In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")
from tqdm.notebook import tqdm
import time
from IPython.display import clear_output

import ee, geemap

In [2]:
%load_ext autoreload
%autoreload 2
import sys  
sys.path.insert(0, '../lib/')

In [3]:
%reload_ext autoreload
import sand_classification as sc
import quality_mosaic as qm

# Part I: Get the locations from Google Drive

In [4]:
sheet_id = '13nF_pJ02Bd70cDJamuKbvZIkIdJ-kOI4O3Cx9K7Wzos'
# sheet_id = '1XqOtPkiE_Q0dfGSoyxrH730RkwrTczcRbDeJJpqRByQ'
sheet_name = 'marchetti'
drive_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
drive_url

'https://docs.google.com/spreadsheets/d/13nF_pJ02Bd70cDJamuKbvZIkIdJ-kOI4O3Cx9K7Wzos/gviz/tq?tqx=out:csv&sheet=marchetti'

In [5]:
output_file_path = '../data/labels/gt-bands.xlsx'

In [6]:
#Read in a google sheets with the ground truth data from string drive_loc
df = pd.read_csv(drive_url)
df

,ID,Latitude,Longitude,Class,Date,Site,Source,Country,Notes,D50,...,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27
0,mar-0,45.509189,8.403156,gravel,2018-09-21,fluvial,https://doi.org/10.1002/esp.5394,Italy,S1,42.40,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,mar-1,45.055640,8.907568,gravel,2018-09-20,fluvial,https://doi.org/10.1002/esp.5394,Italy,P1,34.90,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,mar-2,45.131101,9.976106,sand,2018-09-20,fluvial,https://doi.org/10.1002/esp.5394,Italy,P2,0.43,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,mar-3,45.002953,10.285669,sand,2018-09-20,fluvial,https://doi.org/10.1002/esp.5394,Italy,P3,0.35,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mar-4,38.136695,16.066178,gravel,2018-10-23,fluvial,https://doi.org/10.1002/esp.5394,Italy,B1,32.80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,mar-5,38.126587,16.141498,gravel,2018-10-24,fluvial,https://doi.org/10.1002/esp.5394,Italy,B2,42.30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
#look for any NaN values in the column 'date' or 'Longitude' or 'Latitude' in df_merged and drop them
df = df.dropna(subset=['Date', 'Longitude', 'Latitude']).reset_index(drop=True)
#drop any columns that have 'Unnamed:' in the column name
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df

,ID,Latitude,Longitude,Class,Date,Site,Source,Country,Notes,D50,D84,Classification choices
0,mar-0,45.509189,8.403156,gravel,2018-09-21,fluvial,https://doi.org/10.1002/esp.5394,Italy,S1,42.40,88.0,GRID definitions for gravel
1,mar-1,45.055640,8.907568,gravel,2018-09-20,fluvial,https://doi.org/10.1002/esp.5394,Italy,P1,34.90,49.0,GRID definitions for gravel
2,mar-2,45.131101,9.976106,sand,2018-09-20,fluvial,https://doi.org/10.1002/esp.5394,Italy,P2,0.43,0.8,NaN
3,mar-3,45.002953,10.285669,sand,2018-09-20,fluvial,https://doi.org/10.1002/esp.5394,Italy,P3,0.35,0.6,NaN
4,mar-4,38.136695,16.066178,gravel,2018-10-23,fluvial,https://doi.org/10.1002/esp.5394,Italy,B1,32.80,71.0,GRID definitions for gravel
5,mar-5,38.126587,16.141498,gravel,2018-10-24,fluvial,https://doi.org/10.1002/esp.5394,Italy,B2,42.30,69.0,GRID definitions for gravel


In [8]:
assert(len(df[df['Date'].isna() | df['Longitude'].isna() | df['Latitude'].isna()]) == 0)

# Part II: Select subset for training

Here we iterate over the df and iteratively pick and adjust the locations we want to keep

In [9]:
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()


In [10]:
output = df.copy()
output

,ID,Latitude,Longitude,Class,Date,Site,Source,Country,Notes,D50,D84,Classification choices
0,mar-0,45.509189,8.403156,gravel,2018-09-21,fluvial,https://doi.org/10.1002/esp.5394,Italy,S1,42.40,88.0,GRID definitions for gravel
1,mar-1,45.055640,8.907568,gravel,2018-09-20,fluvial,https://doi.org/10.1002/esp.5394,Italy,P1,34.90,49.0,GRID definitions for gravel
2,mar-2,45.131101,9.976106,sand,2018-09-20,fluvial,https://doi.org/10.1002/esp.5394,Italy,P2,0.43,0.8,NaN
3,mar-3,45.002953,10.285669,sand,2018-09-20,fluvial,https://doi.org/10.1002/esp.5394,Italy,P3,0.35,0.6,NaN
4,mar-4,38.136695,16.066178,gravel,2018-10-23,fluvial,https://doi.org/10.1002/esp.5394,Italy,B1,32.80,71.0,GRID definitions for gravel
5,mar-5,38.126587,16.141498,gravel,2018-10-24,fluvial,https://doi.org/10.1002/esp.5394,Italy,B2,42.30,69.0,GRID definitions for gravel


In [11]:
%reload_ext autoreload
import sand_classification as sc
import quality_mosaic as qm
import file_management as fm

In [12]:
INDEX = 0

In [13]:
sampling_buffer_m = 5
Map = geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [26]:
#Uncomment to change the date of the image
# output['Date'].iloc[INDEX] = '2022-10-14'
# output['ID'].iloc[INDEX] = 'ns-49'

## Manual Loop

Keep running the following 2 cells until you're done with the manual sampling.


In [17]:
s1_s2, sample = qm.get_s1s2_data(output, Map, INDEX, display_smap=False, mosaic_method='median', sampling_buffer_m=sampling_buffer_m, 
                                 max_search_window_months=3, median_samples=5,roi_buffer_m=5000, obia=True)

Index:  0  ID:  mar-0 Class:  gravel  Site:  fluvial
Search window from 2018-06-21 to 2018-12-21


## Use this immediately after the map is refreshed to either select or discard the sample

INDEX will auto update on calling this function.
It will also overwrite just the sheet if the same exists, but will preserve the other sheets in the book

In [15]:
output, INDEX = qm.get_training_sample(output, s1_s2, sample, Map, INDEX, display_clusters=True, obia=True)
fm.write_to_excel(output, output_file_path, sheet_name)
with pd.ExcelWriter(output_file_path,engine="openpyxl",mode="a",if_sheet_exists="replace") as writer:
    output.to_excel(writer,sheet_name=sheet_name,index=False)
output.iloc[INDEX-1]

Original marker accepted
Kept Observation


## Final Output

In [16]:
output

,ID,Latitude,Longitude,Class,Date,Site,Source,Country,Notes,D50,...,B11_mean,B12_mean,VV_mean,VH_mean,mTGSI_mean,BSI_mean,NDWI_mean,keep,location_tweaked,class_code
0,mar-0,45.509189,8.403156,gravel,2018-09-21,fluvial,https://doi.org/10.1002/esp.5394,Italy,S1,42.40,...,937.176147,713.528503,-13.072577,-20.680775,0.061174,0.161584,0.06399,True,False,2
1,mar-1,45.055640,8.907568,gravel,2018-09-20,fluvial,https://doi.org/10.1002/esp.5394,Italy,P1,34.90,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,99
2,mar-2,45.131101,9.976106,sand,2018-09-20,fluvial,https://doi.org/10.1002/esp.5394,Italy,P2,0.43,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,99
3,mar-3,45.002953,10.285669,sand,2018-09-20,fluvial,https://doi.org/10.1002/esp.5394,Italy,P3,0.35,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,99
4,mar-4,38.136695,16.066178,gravel,2018-10-23,fluvial,https://doi.org/10.1002/esp.5394,Italy,B1,32.80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,99
5,mar-5,38.126587,16.141498,gravel,2018-10-24,fluvial,https://doi.org/10.1002/esp.5394,Italy,B2,42.30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,99
